In [1]:
##from transformers import BertTokenizer, BertForMaskedLM
import torch

/home/hyunkyung_lee/.local/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import AutoModel, AutoTokenizer, AutoModelForMaskedLM, AdamW

tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")#, use_fast=False)
model = AutoModelForMaskedLM.from_pretrained("klue/bert-base")

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
###model = AutoModelForMaskedLM.from_pretrained('spanbert-base-cased')


# model_name = "SpanBERT/spanbert-base-cased" 


# # Download pytorch model
# model = AutoModel.from_pretrained(model_name)

In [4]:
corpus = []
corpus_mun = []
corpus_ban = []

with open('train_sample.tsv', 'r') as fp:
    text = fp.read().split('\n')
    
for line in text:
    sentence = line.split('\t')[0]
    corpus.append(sentence)

In [5]:
text[196326]

'2016년에 갤럭시 S7 에지가 폭발한 사건은 어느 지역에서 일어났는가?\t0'

In [6]:
text[196327]

'중국에서 아파트에서 추락하던 3세 아이를 살리고 자신은 혼수상태에 빠진 사람은 누구야?\t1'

In [7]:
text[196328]

'천중핑씨가 추락하는 아이를 구하고 뇌출혈로 인한 의식불명 상태에 빠진 건 언제야?\t1'

In [8]:
corpus_mun = corpus[:196327]
corpus_ban = corpus[196327:]

In [9]:
print(len(corpus_mun))
print(len(corpus_ban))

196327
212423


In [10]:
# 1번 코퍼스 학습 (1. 문어체 수행)

###inputs = tokenizer(corpus_mun, return_tensors='pt', max_length=128, truncation=True, padding='max_length')

In [11]:
# 2번 코퍼스 학습 (2. 반말체 수행)

inputs = tokenizer(corpus_ban, return_tensors='pt', max_length=128, truncation=True, padding='max_length')

In [12]:
inputs

{'input_ids': tensor([[    2,  3693, 27135,  ...,     0,     0,     0],
        [    2,  1652,  2284,  ...,     0,     0,     0],
        [    2,  1652,  2284,  ...,     0,     0,     0],
        ...,
        [    2,  4631,  2145,  ...,     0,     0,     0],
        [    2,  3907,  2073,  ...,     0,     0,     0],
        [    2,     3,     0,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 0,  ..., 0, 0, 0]])}

In [13]:
inputs['labels'] = inputs.input_ids.detach().clone()

In [14]:
inputs

{'input_ids': tensor([[    2,  3693, 27135,  ...,     0,     0,     0],
        [    2,  1652,  2284,  ...,     0,     0,     0],
        [    2,  1652,  2284,  ...,     0,     0,     0],
        ...,
        [    2,  4631,  2145,  ...,     0,     0,     0],
        [    2,  3907,  2073,  ...,     0,     0,     0],
        [    2,     3,     0,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 0,  ..., 0, 0, 0]]), 'labels': tensor([[    2,  3693, 27135,  ...,     0,     0,     0],
        [    2,  1652,  2284,  ...,     0,     0,     0],
        [    2,  1652, 

In [15]:
rand = torch.rand(inputs.input_ids.shape)
rand.shape

torch.Size([212423, 128])

In [16]:
rand

tensor([[0.6347, 0.1933, 0.3486,  ..., 0.1809, 0.6683, 0.3891],
        [0.8342, 0.3585, 0.8035,  ..., 0.1834, 0.0858, 0.4161],
        [0.9794, 0.5001, 0.1759,  ..., 0.2517, 0.6121, 0.5907],
        ...,
        [0.6276, 0.2901, 0.3554,  ..., 0.4817, 0.5533, 0.6876],
        [0.9203, 0.3023, 0.4187,  ..., 0.1850, 0.7664, 0.8519],
        [0.8640, 0.0545, 0.6688,  ..., 0.9607, 0.0702, 0.9018]])

In [17]:
mask_arr = rand < 0 #0.15
mask_arr

tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]])

In [18]:
#inputs.input_ids != 2

In [19]:
#tokenizer

# - MASKING 방법에 변화 주기

In [20]:
corpus_mun[0]

"서울과 충북 괴산에서 '국제 청소년포럼'을 여는 곳은?"

In [21]:
tokenizer.encode(corpus_mun[0])

[2,
 3671,
 2145,
 7249,
 25859,
 27135,
 11,
 3854,
 4857,
 2208,
 2731,
 11,
 1498,
 1428,
 2259,
 601,
 2073,
 35,
 3]

tokenizer.convert_ids_to_tokens(encoded)

tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(encoded))

tokenizer.convert_tokens_to_ids(tokenizer.convert_ids_to_tokens(encoded))

In [22]:
encoded = tokenizer.encode(corpus_mun[0]) # str() 타입의 문장을 인풋
print('*** 토큰화 결과 :',tokenizer.convert_ids_to_tokens(encoded), end='\n\n')
print('*** <문장으로 보기> :', tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(encoded)), end='\n\n')
print('*** 정수 인코딩 :',encoded, end='\n\n') # encoded.ids
print('*** 디코딩 :',tokenizer.decode(encoded), end='\n\n') # encoded.ids

*** 토큰화 결과 : ['[CLS]', '서울', '##과', '충북', '괴산', '##에서', "'", '국제', '청소년', '##포', '##럼', "'", '을', '여', '##는', '곳', '##은', '?', '[SEP]']

*** <문장으로 보기> : [CLS] 서울과 충북 괴산에서'국제 청소년포럼'을 여는 곳은? [SEP]

*** 정수 인코딩 : [2, 3671, 2145, 7249, 25859, 27135, 11, 3854, 4857, 2208, 2731, 11, 1498, 1428, 2259, 601, 2073, 35, 3]

*** 디코딩 : [CLS] 서울과 충북 괴산에서'국제 청소년포럼'을 여는 곳은? [SEP]



In [23]:
inputs.input_ids[0]

tensor([    2,  3693, 27135,  4155, 27135,  8494,  2205,  2414,    23,  2103,
         3651,  2138,  6063,  2088,  3638,  2073, 19666,  2290,  2260,  2170,
         6830,  3611,  2073,  4061,  2275,    35,     3,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [24]:
print(len(tokenizer.encode(corpus_mun[0]))) # 총 19개 토큰
print(len(inputs.input_ids[0])) # 전체 길이는 max_len = 128, 그중 패딩이 아닌 토큰은 총 19개

19
128


In [25]:
encoded = tokenizer.encode('티베로의 가장 큰 장점은 무엇인가?') # str() 타입의 문장을 인풋
print('*** 토큰화 결과 :',tokenizer.convert_ids_to_tokens(encoded), end='\n\n')
print('*** <문장으로 보기> :', tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(encoded)), end='\n\n')
print('*** 정수 인코딩 :',encoded, end='\n\n') # encoded.ids
print('*** 디코딩 :',tokenizer.decode(encoded), end='\n\n') # encoded.ids

*** 토큰화 결과 : ['[CLS]', '티', '##베', '##로', '##의', '가장', '큰', '장점', '##은', '무엇', '##인', '##가', '?', '[SEP]']

*** <문장으로 보기> : [CLS] 티베로의 가장 큰 장점은 무엇인가? [SEP]

*** 정수 인코딩 : [2, 1819, 2472, 2200, 2079, 3676, 1751, 5472, 2073, 3890, 2179, 2116, 35, 3]

*** 디코딩 : [CLS] 티베로의 가장 큰 장점은 무엇인가? [SEP]



In [26]:
print(len(tokenizer.encode('티베로의 가장 큰 장점은 무엇인가?'))) # 총 14개 토큰
###print(len(inputs.input_ids[0])) # 전체 길이는 max_len = 128, 그중 패딩이 아닌 토큰은 총 14개

14


In [27]:
encoded = tokenizer.encode('티베로의 가장 큰 장점은 뭐야?') # str() 타입의 문장을 인풋
print('*** 토큰화 결과 :',tokenizer.convert_ids_to_tokens(encoded), end='\n\n')
print('*** <문장으로 보기> :', tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(encoded)), end='\n\n')
print('*** 정수 인코딩 :',encoded, end='\n\n') # encoded.ids
print('*** 디코딩 :',tokenizer.decode(encoded), end='\n\n') # encoded.ids

*** 토큰화 결과 : ['[CLS]', '티', '##베', '##로', '##의', '가장', '큰', '장점', '##은', '뭐', '##야', '?', '[SEP]']

*** <문장으로 보기> : [CLS] 티베로의 가장 큰 장점은 뭐야? [SEP]

*** 정수 인코딩 : [2, 1819, 2472, 2200, 2079, 3676, 1751, 5472, 2073, 1097, 2275, 35, 3]

*** 디코딩 : [CLS] 티베로의 가장 큰 장점은 뭐야? [SEP]



## - 연습

In [28]:
encoded = tokenizer.encode('[MASK]') # str() 타입의 문장을 인풋
print('*** 토큰화 결과 :',tokenizer.convert_ids_to_tokens(encoded), end='\n\n')
print('*** <문장으로 보기> :', tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(encoded)), end='\n\n')
print('*** 정수 인코딩 :',encoded, end='\n\n') # encoded.ids
print('*** 디코딩 :',tokenizer.decode(encoded), end='\n\n') # encoded.ids

*** 토큰화 결과 : ['[CLS]', '[MASK]', '[SEP]']

*** <문장으로 보기> : [CLS] [MASK] [SEP]

*** 정수 인코딩 : [2, 4, 3]

*** 디코딩 : [CLS] [MASK] [SEP]



In [29]:
encoded = tokenizer.encode('서울과 충북 괴산에서 \'국제 청소년포럼\'을 여는 곳은?') # str() 타입의 문장을 인풋
encoded[-3] = 4 # '[MASK]' 토큰
encoded[-4] = 4 # '[MASK]' 토큰

print('*** 토큰화 결과 :',tokenizer.convert_ids_to_tokens(encoded), end='\n\n')
print('*** <문장으로 보기> :', tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(encoded)), end='\n\n')
print('*** 정수 인코딩 :',encoded, end='\n\n') # encoded.ids
print('*** 디코딩 :',tokenizer.decode(encoded), end='\n\n') # encoded.ids

*** 토큰화 결과 : ['[CLS]', '서울', '##과', '충북', '괴산', '##에서', "'", '국제', '청소년', '##포', '##럼', "'", '을', '여', '##는', '[MASK]', '[MASK]', '?', '[SEP]']

*** <문장으로 보기> : [CLS] 서울과 충북 괴산에서'국제 청소년포럼'을 여는 [MASK] [MASK]? [SEP]

*** 정수 인코딩 : [2, 3671, 2145, 7249, 25859, 27135, 11, 3854, 4857, 2208, 2731, 11, 1498, 1428, 2259, 4, 4, 35, 3]

*** 디코딩 : [CLS] 서울과 충북 괴산에서'국제 청소년포럼'을 여는 [MASK] [MASK]? [SEP]



In [30]:
# mask_arr = (rand < 0.15) * (inputs.input_ids != 2) * (inputs.input_ids != 3)
# mask_arr

In [31]:
mask_arr

tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]])

In [32]:
inputs.input_ids[0].nonzero()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  """Entry point for launching an IPython kernel.


tensor([[ 0],
        [ 1],
        [ 2],
        [ 3],
        [ 4],
        [ 5],
        [ 6],
        [ 7],
        [ 8],
        [ 9],
        [10],
        [11],
        [12],
        [13],
        [14],
        [15],
        [16],
        [17],
        [18],
        [19],
        [20],
        [21],
        [22],
        [23],
        [24],
        [25],
        [26]])

In [33]:
torch.flatten(inputs.input_ids[0].nonzero()).tolist()

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26]

In [34]:
corpus_mun[0]

"서울과 충북 괴산에서 '국제 청소년포럼'을 여는 곳은?"

In [35]:
print(len(corpus_mun[0]))
print(len(torch.flatten(mask_arr[0].nonzero()).tolist()))

30
0


In [36]:
selection = []

for i in range(mask_arr.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )

In [37]:
selection[:10]

[[], [], [], [], [], [], [], [], [], []]

In [38]:
inputs.input_ids[0]

tensor([    2,  3693, 27135,  4155, 27135,  8494,  2205,  2414,    23,  2103,
         3651,  2138,  6063,  2088,  3638,  2073, 19666,  2290,  2260,  2170,
         6830,  3611,  2073,  4061,  2275,    35,     3,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [39]:
tokenizer.decode(inputs.input_ids[0])

'[CLS] 중국에서 아파트에서 추락하던 3세 아이를 살리고 자신은 혼수상태에 빠진 사람은 누구야? [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [40]:
# for i in range(mask_arr.shape[0]):
#     inputs.input_ids[i, selection[i]] = 3 # [SEP] 토큰

In [41]:
inputs.input_ids[0]

tensor([    2,  3693, 27135,  4155, 27135,  8494,  2205,  2414,    23,  2103,
         3651,  2138,  6063,  2088,  3638,  2073, 19666,  2290,  2260,  2170,
         6830,  3611,  2073,  4061,  2275,    35,     3,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

# - 어텐션 마스크 변형주기

In [42]:
inputs.attention_mask[0]

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])

In [43]:
inputs.token_type_ids[0]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])

In [44]:
type(inputs.attention_mask[0])

torch.Tensor

In [45]:
inputs.attention_mask.nonzero()[0]

tensor([0, 0])

- 1. 문어체 학습(fine-tuning)용

In [46]:
# (사전 계산) 각 문장들의 토큰 길이 구해놓기
# corpus_size = len(corpus_mun)

# max_len_list = []

# for idx in range(corpus_size):
#     encoded = tokenizer.encode(corpus_mun[idx])
#     max_len_list.append(len(encoded))
    
# max_len_list

- 2. 반말체 학습(fine-tuning)용

In [47]:
corpus_size = len(corpus_ban)

max_len_list = []

for idx in range(corpus_size):
    encoded = tokenizer.encode(corpus_ban[idx])
    max_len_list.append(len(encoded))
    
max_len_list

[27,
 26,
 19,
 22,
 11,
 16,
 15,
 16,
 14,
 16,
 16,
 16,
 16,
 14,
 24,
 20,
 34,
 22,
 35,
 11,
 12,
 10,
 13,
 11,
 9,
 21,
 22,
 23,
 22,
 24,
 26,
 27,
 13,
 51,
 31,
 48,
 16,
 10,
 25,
 11,
 13,
 8,
 17,
 14,
 46,
 21,
 15,
 22,
 18,
 17,
 17,
 34,
 22,
 18,
 10,
 18,
 19,
 24,
 10,
 17,
 13,
 15,
 12,
 13,
 30,
 15,
 20,
 11,
 13,
 10,
 16,
 21,
 14,
 27,
 21,
 8,
 14,
 13,
 12,
 12,
 16,
 17,
 14,
 18,
 16,
 19,
 77,
 24,
 31,
 14,
 23,
 23,
 22,
 45,
 27,
 22,
 15,
 23,
 12,
 14,
 15,
 11,
 16,
 12,
 35,
 14,
 11,
 17,
 11,
 17,
 15,
 43,
 40,
 25,
 31,
 20,
 15,
 21,
 18,
 13,
 13,
 20,
 11,
 29,
 11,
 10,
 11,
 10,
 13,
 13,
 24,
 10,
 19,
 14,
 9,
 10,
 17,
 23,
 23,
 26,
 22,
 19,
 18,
 14,
 17,
 17,
 21,
 13,
 17,
 15,
 13,
 28,
 9,
 16,
 24,
 22,
 25,
 26,
 13,
 15,
 14,
 28,
 25,
 18,
 18,
 14,
 14,
 17,
 34,
 25,
 20,
 27,
 17,
 13,
 7,
 44,
 10,
 14,
 34,
 15,
 12,
 13,
 18,
 25,
 18,
 14,
 15,
 14,
 21,
 11,
 16,
 18,
 17,
 16,
 17,
 14,
 22,
 16,
 19,
 22,
 28,
 

In [48]:
# selection = []

# for i in range(mask_arr.shape[0]):
#     selection.append(
#         torch.flatten(mask_arr[i].nonzero()).tolist()
#     )

In [49]:
mask_arr

tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]])

In [50]:
mask_arr[0][19] = True

In [51]:
mask_arr[0] # 변화 적용됨을 확인함 !

tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,  True,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False])

In [52]:
mask_arr[0][19-2 : 19+4] = True

In [53]:
mask_arr[0]

tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False,  True,  True,  True,
         True,  True,  True, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False])

In [54]:
mask_arr.shape

torch.Size([212423, 128])

In [55]:
inputs

{'input_ids': tensor([[    2,  3693, 27135,  ...,     0,     0,     0],
        [    2,  1652,  2284,  ...,     0,     0,     0],
        [    2,  1652,  2284,  ...,     0,     0,     0],
        ...,
        [    2,  4631,  2145,  ...,     0,     0,     0],
        [    2,  3907,  2073,  ...,     0,     0,     0],
        [    2,     3,     0,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 0,  ..., 0, 0, 0]]), 'labels': tensor([[    2,  3693, 27135,  ...,     0,     0,     0],
        [    2,  1652,  2284,  ...,     0,     0,     0],
        [    2,  1652, 

In [56]:
print(inputs.input_ids[0, 0:19-4])
print(inputs.attention_mask[0, 0:19-4])
print()
print(inputs.input_ids[0, 19-4:19+4])
print(inputs.attention_mask[0, 19-4:19+4])

tensor([    2,  3693, 27135,  4155, 27135,  8494,  2205,  2414,    23,  2103,
         3651,  2138,  6063,  2088,  3638])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

tensor([ 2073, 19666,  2290,  2260,  2170,  6830,  3611,  2073])
tensor([1, 1, 1, 1, 1, 1, 1, 1])


In [57]:
## 확인
inputs.input_ids[0]

tensor([    2,  3693, 27135,  4155, 27135,  8494,  2205,  2414,    23,  2103,
         3651,  2138,  6063,  2088,  3638,  2073, 19666,  2290,  2260,  2170,
         6830,  3611,  2073,  4061,  2275,    35,     3,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [58]:
max_len_list[0] # [SEP] 토큰은 현재 26에 있음

27

- 'attention_mask'에 변화주는 파트

In [59]:
# ########## 중요 !! ##########
# for i in range(corpus_size):
#     leng = max_len_list[i]
#     inputs.attention_mask[i, 0:leng-4] = 0 
#     inputs.attention_mask[i, leng-4:leng+4] = 1 # 1 로 바꿈 (masking해서 가르칠 대상)    
    
#     ##### 추가 #####
#     inputs.token_type_ids[i, leng+4:] = 1 # 뒤는 두 번째 문장인 것처럼 인식시켜서 -> 현재 문장의 끝을 leng+3 까지로 여기도록

In [60]:
########## 중요 !! ##########
for i in range(corpus_size):
    leng = max_len_list[i]
    inputs.attention_mask[i, 0:leng-5] = 0 
    inputs.attention_mask[i, leng-5:leng+1] = 1 # 1 로 바꿈 (masking해서 가르칠 대상) 
    
    ##### 추가 #####
    inputs.token_type_ids[i, leng:] = 1 # 뒤는 두 번째 문장인 것처럼 인식시켜서 -> 현재 문장의 끝을 leng+3 까지로 여기도록
    
    ##### 추가 #####
    inputs.input_ids[i, leng-1] = 0 # [SEP] 토큰을 [PAD] 토큰으로 바꿈

In [61]:
print(inputs.input_ids[0, 0:19-4])
print(inputs.attention_mask[0, 0:19-4])
print()
print(inputs.input_ids[0, 19-4:19+4])
print(inputs.attention_mask[0, 19-4:19+4])

tensor([    2,  3693, 27135,  4155, 27135,  8494,  2205,  2414,    23,  2103,
         3651,  2138,  6063,  2088,  3638])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

tensor([ 2073, 19666,  2290,  2260,  2170,  6830,  3611,  2073])
tensor([0, 0, 0, 0, 0, 0, 0, 1])


In [62]:
inputs.attention_mask[0]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
        1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])

In [63]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [64]:
dataset = Dataset(inputs)

In [65]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True) ###

In [66]:
##device = torch.device('cuda:0')
torch.cuda.is_available()

True

In [67]:
device = ('cuda:0')

model.to(device)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [68]:
model.train()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [69]:
optim = AdamW(model.parameters(), lr=1e-5)

from tqdm import tqdm

epochs = 5 ####
for epoch in range(epochs):
    loop = tqdm(dataloader, leave=True)
    for batch in loop:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        ### 추가
        token_type_ids = batch['token_type_ids'].to(device)

        
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels, token_type_ids=token_type_ids)
        loss = outputs.loss
        loss.backward()
        optim.step()
        
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())
        
    
    print(epoch)
    print(loss)
    print('* * * * *')
    ### 1. 문어체 학습 시
    #PATH = 'model/segment_padded_MLM_train_mun_{0}_epoch'.format(epoch)
    
    ### 2. 반말체 학습 시
    PATH = 'model/small_segment_padded_MLM_train_ban_{0}_epoch'.format(epoch)
    torch.save(model, PATH)

/home/hyunkyung_lee/.local/lib/python3.6/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
  0%|          | 0/6639 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """
Epoch 0: 100%|██████████| 6639/6639 [36:01<00:00,  3.07it/s, loss=5.59e-5] 


0
tensor(5.5897e-05, device='cuda:0', grad_fn=<NllLossBackward>)
* * * * *


Epoch 1: 100%|██████████| 6639/6639 [36:09<00:00,  3.06it/s, loss=1.33e-5] 


1
tensor(1.3280e-05, device='cuda:0', grad_fn=<NllLossBackward>)
* * * * *


Epoch 2: 100%|██████████| 6639/6639 [36:10<00:00,  3.06it/s, loss=4.84e-6] 


2
tensor(4.8410e-06, device='cuda:0', grad_fn=<NllLossBackward>)
* * * * *


Epoch 3: 100%|██████████| 6639/6639 [36:10<00:00,  3.06it/s, loss=1.42e-6] 


3
tensor(1.4223e-06, device='cuda:0', grad_fn=<NllLossBackward>)
* * * * *


Epoch 4: 100%|██████████| 6639/6639 [36:10<00:00,  3.06it/s, loss=1.55e-6]


4
tensor(1.5471e-06, device='cuda:0', grad_fn=<NllLossBackward>)
* * * * *
